In [51]:
from os import getcwd, listdir, makedirs
from os.path import join, exists, isfile, isdir, abspath, pardir
from json import load, dump
import pandas as pd
import numpy as np
import geopandas as gp
from shapely.geometry import Point, LineString

##### Configs

In [2]:
shape_files_dir = join(abspath(join(getcwd(), pardir)), "2109_STIB_MIVB_Network")
line_shape_file = join(shape_files_dir, "ACTU_LINES.shp")
stops_shape_file = join(shape_files_dir, "ACTU_STOPS.shp")

In [3]:
line_df = gp.read_file(line_shape_file)
stops_df = gp.read_file(stops_shape_file)

#### For bus 95

In [4]:
transport = '095b'

In [5]:
lines = line_df.loc[line_df['LIGNE'].isin([transport])]
stops = stops_df.loc[stops_df['Code_Ligne'].isin([transport])]

In [6]:
lines_v1 = lines.loc[lines['VARIANTE'].isin([1])]
lines_v2 = lines.loc[lines['VARIANTE'].isin([2])]
stops_v1 = stops.loc[stops['Variante'].isin([1])]
stops_v2 = stops.loc[stops['Variante'].isin([2])]

In [26]:
v1_points = list(lines_v1['geometry'])[0]
v2_points = list(lines_v2['geometry'])[0]

In [10]:
# v1_points = [Point(x, y) for x, y, _ in list(v1_points.coords)]
# v2_points = [Point(x, y) for x, y, _ in list(v2_points.coords)]

In [17]:
stops_data_v1 = dict(zip(stops_v1.stop_id, stops_v1.geometry))
stops_data_v2 = dict(zip(stops_v2.stop_id, stops_v2.geometry))

In [27]:
type(v1_points)

shapely.geometry.linestring.LineString

In [46]:
type(stops_data_v1)

dict

In [32]:
v1_points.project(stops_data_v1['1128'])

701.1202884557507

In [52]:
def calculate_distance_between_stops(stops_data, points):
    if not isinstance(stops_data, dict): raise ValueError("Kindly, pass the stops data with 'stop_id' and 'stop_points'.")
    if not isinstance(points, LineString): raise ValueError("Kindly, pass the correct points data.")
    previous_stop_id, previous_distance, distance = None, None, list()
    for index, stop_id in enumerate(stops_data):
        stop_point = stops_data[stop_id]
        stop_distance = points.project(stop_point)
        if index != 0:
            v = {"from": previous_stop_id, "to": stop_id, "distance": float(stop_distance - previous_distance) }
            distance.append(v)
        previous_stop_id = stop_id
        previous_distance = stop_distance
    return distance

In [57]:
data = calculate_distance_between_stops(stops_data=stops_data_v1, points=v1_points)

In [ ]:
calculate_distance_between_stops(stops_data=stops_data_v2, points=v2_points)

In [58]:
def __calculate_distance_between_stops__(stops_data=dict(), points=list()):
    if not isinstance(stops_data, dict): raise ValueError("Kindly, pass the stops data with 'stop_id' and 'stop_points'.")
    if not isinstance(points, LineString): raise ValueError("Kindly, pass the correct points data.")
    
    def calculate_minimum_distance(stop_point, line_points):
        minimum_distance = None
        minimum_sum_distance = None
        minimum_point = None
        sum_distance = float(0)
        line_points = list(line_points.coords)
        for index, line_point in enumerate(line_points):
            x, y, _ = line_point
            point = Point(x, y)
            distance = point.distance(stop_point)
            sum_distance = sum_distance + distance
            if (minimum_distance is None) or (distance < minimum_distance):
                minimum_distance = distance
                minimum_sum_distance = sum_distance
                minimum_point = point
        return minimum_point, minimum_sum_distance
    
    def get_distance_from_point(stop_point, line_points, start_index=0):
        sum_distance = float(0)
        for index, line_point in enumerate(line_points[start_index:]):
            x, y, _ = line_point
            point = Point(x, y)
            if point.equals(stop_point):
                return index + start_index, sum_distance
            else:
                sum_distance = sum_distance + point.distance(stop_point)
        
    data = list()
    stops = list(stops_data.keys())
    
    for stop in stops:
        stop_point = stops_data[stop]
        min_point, distance_from_start = calculate_minimum_distance(stop_point, points)
        value = { "stop": stop, "stop_point": stop_point, "min_point": min_point, "distance_from_start": distance_from_start }
        data.append(value)
        
#     stop_index = 0
#     for i, d in enumerate(stops_data):
#         distance_from_nearest_stop = d['stop_point'].distance(d['min_point'])
#         stops_data[i]['distance_from_nearest_stop'] = distance_from_nearest_stop
#         stop_index, distance = get_distance_from_point(stop_point= d['stop_point'], line_points= line_points, start_index=stop_index)
#         stops_data[i]['distance'] = distance
        
    return data
_data = __calculate_distance_between_stops__(stops_data=stops_data_v1, points=v1_points)

In [61]:
def search(d, k, v):
    for p in d:
        if p[k] == v:
            return p

In [74]:
stop = '2268'

In [75]:
d1 = search(_data, 'stop', stop)
d2 = search(data, 'to', stop)

In [76]:
d1, d2

({'stop': '2268',
  'stop_point': <shapely.geometry.point.Point at 0x7f78b8ae4af0>,
  'min_point': <shapely.geometry.point.Point at 0x7f78b82b6460>,
  'distance_from_start': 2140.3334461594673},
 {'from': '2278', 'to': '2268', 'distance': 233.8455980071548})

In [77]:
d2['distance']

233.8455980071548

In [78]:
v1_points.project(d1['min_point'])

233.84091107783337

In [79]:
d1['stop_point'].distance(d1['min_point']) + d2['distance']

236.93428705205608

In [ ]:
d = _data[10]
d['stop_point'].distance(d['min_point'])

In [ ]:
d

In [ ]:
points = [ Point(x, y) for x, y, _ in all_points ]

In [ ]:
# type(all_points)
v1 = list(b95_lines_v1['geometry'])[0]

In [ ]:
v1.project(d['min_point'])

In [ ]:
for d in _data:
    print("Stop: {}\tMin. distance: {}\tTotal Distance: {}".format(d['stop'], d['stop_point'].distance(d['min_point']), v1.project(d['min_point'])))
    # print(d['stop_point'].distance(d['min_point']))
    

In [ ]:
p1 = _data[0]['stop_point']
p2 = _data[0]['min_point']

In [ ]:
p1.equals(p1)